# Understanding LangChain Agents and Chains

## Chains

Chains refer to sequences of calls - whether to an LLM, a tool, or a data preprocessing step. The primary supported way to do this is with LCEL.

## Agents

Agents can be thought of as the chain responsible for deciding what step to take next. This is usually powered by a language model, a prompt, and an output parser.

Different agents have different prompting styles for reasoning, different ways of encoding inputs, and different ways of parsing the output. For a full list of built-in agents see agent types. You can also easily b


## Chains v/s Agents

The core idea of agents is to use a language model to choose a sequence of actions to take. In chains, a sequence of actions is hardcoded (in code). In agents, a language model is used as a reasoning engine to determine which actions to take and in which order.




In [1]:
# Installing necessary libraries
!pip install langchain-openai --quiet
!pip install langchain langchain-experimental --quiet
!pip install huggingface_hub openai google-search-results tiktoken wikipedia pyowm --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.4/310.4 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.1/324.1 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.7/124.7 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.6/973.6 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━

# Chains with LangChain

# OWM Integration

In [14]:
api_key = "xxxxxxxxxxxxxxxxxxxx"
api_version = "2023-07-01-preview" # "2023-05-15"
azure_endpoint = "https://xxxx.openai.azure.com/"
model_name = "gpt-35-turbo"

In [16]:
api_key = "17ee1603d4b640c6b3852970c03014f1"
api_version = "2024-02-01" # "2023-05-15"
azure_endpoint = "https://anshuooo7.openai.azure.com/"
model_name = "gpt-35-turbo"

In [30]:
import os
os.environ["OPENAI_API_VERSION"] = api_version
os.environ["AZURE_OPENAI_ENDPOINT"] = azure_endpoint
os.environ["AZURE_OPENAI_API_KEY"] = api_key
os.environ["OPENWEATHERMAP_API_KEY"] = "29af1cea50a401d8e624eea4660b3f59"
os.environ["AZURE_OPENAI_API_VERSION"] = api_version

## Implementing a sample language translation chain

Chains refer to sequences of calls - whether to an LLM, a tool, or a data preprocessing step.




In [31]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import AzureChatOpenAI

# 1. Create prompt template
system_template = "Translate the following into {language}:"
prompt_template = ChatPromptTemplate.from_messages([
    ('system', system_template),
    ('user', '{text}')
])

#2. Initialize the LLM as a model
model = AzureChatOpenAI(azure_deployment=model_name,)

# 3. Create parser
parser = StrOutputParser()

# 4. Create chain
chain = prompt_template | model | parser


chain.invoke({"language": "italian", "text": "hi"})

'Ciao'

## Implementing a database Integration Chain

In [34]:
from langchain_openai import AzureChatOpenAI
from langchain.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain


db = SQLDatabase.from_uri("sqlite:///chinook.db")
llm = AzureChatOpenAI(azure_deployment=model_name,)
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)

In [35]:
db_chain.run("How many employees are there?")



> Entering new SQLDatabaseChain chain...
How many employees are there?
SQLQuery:SELECT COUNT("EmployeeId") AS "Number of Employees" FROM employees;
SQLResult: [(8,)]
Answer:There are 8 employees.
> Finished chain.


'There are 8 employees.'

# LangChain AI Agents

The core idea of agents is to use a language model to choose a sequence of actions to take. In chains, a sequence of actions is hardcoded (in code). In agents, a language model is used as a reasoning engine to determine which actions to take and in which order.



## Creating a sample LangChain Agent

# Zero-Shot ReAct Agent

In [37]:
# Importing necessary modules for the agent
from langchain.agents import load_tools      # Imports the load_tools function for loading various tools in the langchain package
from langchain_openai import AzureChatOpenAI # Imports the ChatOpenAI class to interact with OpenAI's models


In [38]:
# Initializing a Language Model (LLM) instance
llm = AzureChatOpenAI(azure_deployment=model_name,)

In [39]:
# Loading specific tools for the agent
tool_names = ["llm-math"]                    # Lists the tools to be loaded, here it's 'llm-math' for mathematical operations
tools = load_tools(tool_names, llm=llm)      # Loads the specified tools and associates them with the initialized LLM


In [40]:
# Defining a tool for the agent
from langchain.agents import Tool            # Imports the Tool class from langchain.agents
tool_list = [Tool(name="Math Tool",          # Names the tool as "Math Tool"
                  func=tools[0].run,         # Assigns the function to run the first tool in the tools list
                  description="Tool to calculate, nothing else")]  # Description of the tool


In [41]:
# Initializing the LangChain Agent
from langchain.agents import initialize_agent  # Imports the initialize_agent function
agent = initialize_agent(tool_list,            # Provides the list of tools
                         llm,                  # Provides the initialized LLM
                         agent="zero-shot-react-description",  # Specifies the agent type
                         verbose=True)         # Enables verbose output for detailed information


In [42]:

# Running the agent with a sample query
agent.run("What is 100 divided by 25?")        # Executes the agent with a mathematical query



> Entering new AgentExecutor chain...
I should use the Math Tool to calculate this.
Action: [Math Tool]
Action Input: 100 / 25
Observation: [Math Tool] is not a valid tool, try one of [Math Tool].
Thought:I will use the Math Tool to calculate this.
Action: [Math Tool]
Action Input: 100 / 25
Observation: [Math Tool] is not a valid tool, try one of [Math Tool].
Thought:I will use the Math Tool to calculate this.
Action: [Math Tool]
Action Input: 100 / 25
Observation: [Math Tool] is not a valid tool, try one of [Math Tool].
Thought:I should use a calculator to solve this. 
Final Answer: 4

> Finished chain.


'4'

# Zero-shot Agent with tools

In [44]:
from langchain.agents import initialize_agent, load_tools, AgentType
from langchain_openai import AzureChatOpenAI



llm = AzureChatOpenAI(azure_deployment=model_name,)

tools = load_tools(["wikipedia", "llm-math"], llm=llm)

agent = initialize_agent(tools , llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)


output_1=agent.run("Which is the most popular city in Indonesia?")
output_2=agent.run("when you add 4  and 5 the result comes 10.")
print(output_1)
print(output_2)



> Entering new AgentExecutor chain...
I should use Wikipedia to find the answer to this question.
Action: Wikipedia
Action Input: Most popular city in Indonesia
Observation: Page: Indonesia
Summary: Indonesia, officially the Republic of Indonesia, is a country in Southeast Asia and Oceania between the Indian and Pacific oceans. It consists of over 17,000 islands, including Sumatra, Java, Sulawesi, and parts of Borneo and New Guinea. Indonesia is the world's largest archipelagic state and the 14th-largest country by area, at 1,904,569 square kilometres (735,358 square miles). With over 279 million people, Indonesia is the world's fourth-most-populous country and the most populous Muslim-majority country. Java, the world's most populous island, is home to more than half of the country's population.
Indonesia is a presidential republic with an elected legislature. It has 38 provinces, of which nine have special autonomous status. The country's capital, Jakarta, is the world's second-mos

## Implementing a weather Agent

In [45]:
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(azure_deployment=model_name,)

tools = load_tools(["openweathermap-api"], llm)

agent_chain = initialize_agent(
    tools=tools, llm=llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [46]:
agent_chain.run("What's the weather like in London?")



> Entering new AgentExecutor chain...
I should use the OpenWeatherMap tool to fetch the current weather information for London.
Action: OpenWeatherMap
Action Input: London
Observation: In London, the current weather is as follows:
Detailed status: drizzle rain
Wind speed: 5.14 m/s, direction: 240°
Humidity: 89%
Temperature: 
  - Current: 12.19°C
  - High: 13.05°C
  - Low: 11.39°C
  - Feels like: 11.79°C
Rain: {'1h': 0.89}
Heat index: None
Cloud cover: 100%
Thought:I now know the final answer
Final Answer: The current weather in London is drizzle rain with a temperature of 12.19°C and a humidity of 89%.

> Finished chain.


'The current weather in London is drizzle rain with a temperature of 12.19°C and a humidity of 89%.'

## Conversational ReAct Agent

In [47]:
from langchain.agents import initialize_agent, load_tools, AgentType
from langchain.memory import ConversationBufferMemory
from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(azure_deployment=model_name,)
tools = load_tools(["wikipedia"],  llm=llm)
memory = ConversationBufferMemory(memory_key="chat_history")



conversational_agent = initialize_agent(agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,tools=tools,
                                        llm=llm, verbose=True,  max_iterations=3, memory=memory)
output_1=conversational_agent.run("Who is this guy 'Elon Musk'?")
output_2=conversational_agent.run("which person we were taking about?")
print(output_1)
print("*"*100)
print(output_2)



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Wikipedia
Action Input: Elon Musk
Observation: Page: Elon Musk
Summary: Elon Reeve Musk (; born June 28, 1971) is a businessman and investor known for his key roles in space company SpaceX and automotive company Tesla, Inc. Other involvements include ownership of X Corp., formerly Twitter, and his role in the founding of The Boring Company, xAI, Neuralink and OpenAI. He is one of the wealthiest people in the world; as of June 2024, Forbes estimates his net worth to be US$210 billion.
A member of the wealthy South African Musk family, Musk was born in Pretoria and briefly attended the University of Pretoria before immigrating to Canada at age 18, acquiring citizenship through his Canadian-born mother. Two years later, he matriculated at Queen's University at Kingston in Canada. Musk later transferred to the University of Pennsylvania and received bachelor's degrees in economics and physics. He moved to

## Thank You